<div>
<img src="https://res-5.cloudinary.com/crunchbase-production/image/upload/c_lpad,h_256,w_256,f_auto,q_auto:eco/v1455514364/pim02bzqvgz0hibsra41.png" align="right"><br><br><FONT FACE="times new roman" SIZE=5>
<b></b>
<br>
<i><b>Docente:</b></i> Camilo Rodriguez.
<br>
<i><b>Asignatura:</b></i> Semillero de investigación
<br>
<i><b>Estudiante:</b></i> Esteban Rodriguez, Laura Gomez y Santiago Niño
<br>
<i><b>Tema:</b></i> TrabajoSemilleroInvestigacion_Taxis
<br>
<br> Octubre/2023
<br>
</FONT>
</div>

#**Importación de librerias**

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum, date_format, min, max, mean, avg, stddev, count, to_timestamp, dayofweek, hour, year
from pyspark.sql import SparkSession
import plotly.express as px
from pyspark.ml.stat import Correlation
import pandas as pd
import numpy as np
import seaborn as sns
import dask.dataframe as dd
import matplotlib.pyplot as plt
from google.colab import drive

#**1. Recopilar y preparar los datos**

In [ ]:
drive.mount('/content/drive')
zip_file_path = "/content/drive/MyDrive/SERVICIO_UNIFICADO_2023.parquet"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Configuración de la sesión Spark
spark = SparkSession.builder \
    .appName("BogotaData") \
    .master("local[*]") \
    .config("spark.executor.memory", "1024g") \
    .config("spark.driver.memory", "68g") \
    .config("spark.executor.cores", "1024") \
    .getOrCreate()

In [ ]:
# Lee el archivo Parquet como DataFrame
df = spark.read.parquet(zip_file_path)

# Muestra el DataFrame
df.show()

+--------+-----+-----------+------------------+------------------+------------------+------------------+----------+------------+-------------------+----------+---------+----+-----+
|      ID| CITY|       USER|       LATITUDEORI|      LATITUDEDEST|      LONGITUDEORI|     LONGITUDEDEST|    STATUS|     CHANNEL|       COMPLETEDATE|      DATE|DAYOFWEEK|HOUR|MONTH|
+--------+-----+-----------+------------------+------------------+------------------+------------------+----------+------------+-------------------+----------+---------+----+-----+
|80206465|11001|  8168434.0| 4.646834566745485| 4.669336700000088|-74.10161945968866|-74.11134894799994| CANCELADO|         APP|2022-10-01 00:00:00|2022-10-01|        6|   0|   10|
|80206466|11001|  8692306.0| 4.648860661223959|              NULL|-74.10982768982649|              NULL|ABANDONADO|         APP|2022-10-01 00:00:00|2022-10-01|        6|   0|   10|
|80206467|11001|1.0186684E7| 4.607888513534026|           4.54708|-74.07884381711483|         -

#2.Preprocesamiento de datos

## 2.1 **Limpieza de datos:**
* Se verifica si se presentan datos no disponibles o NaN para eliminarlos.
* Se eliminan las variables que no se usarán en el desarrollo.
*Se convierte los datos de tipo objeto al tipo de dato que si corresponda.

In [ ]:
#cuantos datos tenemos de bogotá
bogota = df.filter(df.CITY == 11001)
bogota.count()

14870616

In [ ]:
bogota.printSchema()

root
 |-- ID: long (nullable = true)
 |-- CITY: long (nullable = true)
 |-- USER: double (nullable = true)
 |-- LATITUDEORI: double (nullable = true)
 |-- LATITUDEDEST: double (nullable = true)
 |-- LONGITUDEORI: double (nullable = true)
 |-- LONGITUDEDEST: double (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- CHANNEL: string (nullable = true)
 |-- COMPLETEDATE: string (nullable = true)
 |-- DATE: string (nullable = true)
 |-- DAYOFWEEK: long (nullable = true)
 |-- HOUR: long (nullable = true)
 |-- MONTH: long (nullable = true)



In [ ]:
# Contar los valores nulos en Bogotá
nulos_bogota = bogota.select([sum(col(column).isNull().cast("int")).alias(column) for column in bogota.columns])
nulos_bogota.show()

+---+----+----+-----------+------------+------------+-------------+------+-------+------------+----+---------+----+-----+
| ID|CITY|USER|LATITUDEORI|LATITUDEDEST|LONGITUDEORI|LONGITUDEDEST|STATUS|CHANNEL|COMPLETEDATE|DATE|DAYOFWEEK|HOUR|MONTH|
+---+----+----+-----------+------------+------------+-------------+------+-------+------------+----+---------+----+-----+
|  0|   0|  91|          0|     2812968|           0|      2812968|     0|      0|           0|   0|        0|   0|    0|
+---+----+----+-----------+------------+------------+-------------+------+-------+------------+----+---------+----+-----+



In [ ]:
# Eliminar registros con valores nulos
bogota = bogota.dropna()

In [ ]:
nulos_bogota = bogota.select([sum(col(column).isNull().cast("int")).alias(column) for column in bogota.columns])
nulos_bogota.show()

+---+----+----+-----------+------------+------------+-------------+------+-------+------------+----+---------+----+-----+
| ID|CITY|USER|LATITUDEORI|LATITUDEDEST|LONGITUDEORI|LONGITUDEDEST|STATUS|CHANNEL|COMPLETEDATE|DATE|DAYOFWEEK|HOUR|MONTH|
+---+----+----+-----------+------------+------------+-------------+------+-------+------------+----+---------+----+-----+
|  0|   0|   0|          0|           0|           0|            0|     0|      0|           0|   0|        0|   0|    0|
+---+----+----+-----------+------------+------------+-------------+------+-------+------------+----+---------+----+-----+



In [ ]:
bogota.count()

12057629

In [ ]:
valores_a_eliminar = ['ABORDOCONFIRMADO', 'CONFIRMADO', 'FINALIZADO', 'ALFRENTE', 'NUEVO', 'RECHAZADO', 'ASIGNADO']

# Elimina las filas que contienen los valores en la columna 'STATUS'
bogota = bogota.filter(~bogota['STATUS'].isin(valores_a_eliminar))

In [ ]:
valores_a_eliminar =['EXTERNO','CALLE','WEB','Reserva','CHAT_BOT_803','CHAT_BOT_689','IONTAXI','Kiosko #1']

# Elimina las filas que contienen los valores en la columna 'STATUS'
bogota = bogota.filter(~bogota['CHANNEL'].isin(valores_a_eliminar))

In [ ]:
bogota.count()

12005856

In [ ]:
# Número de filas en el DataFrame 'bogota_filtrado'
filas = bogota.count()

# Número de columnas en el DataFrame 'bogota_filtrado'
columnas  = len(bogota.columns)

In [ ]:
print(f"Número de filas en Bogotá filtrado: {filas}")
print(f"Número de columnas en Bogotá filtrado: {columnas}")

Número de filas en Bogotá filtrado: 12005856
Número de columnas en Bogotá filtrado: 14


In [ ]:
# Calcula el valor mínimo y máximo de la columna 'COMPLETEDATE'
min_date = bogota.selectExpr("min(COMPLETEDATE) as min_date").collect()[0]["min_date"]
max_date = bogota.selectExpr("max(COMPLETEDATE) as max_date").collect()[0]["max_date"]

# Imprime el valor mínimo y máximo
print("Valor mínimo de COMPLETEDATE:", min_date)
print("Valor máximo de COMPLETEDATE:", max_date)

Valor mínimo de COMPLETEDATE: 2022-10-01 00:00:00
Valor máximo de COMPLETEDATE: 2023-04-20 13:25:47


Se tiene registro desde el primero de octubre de 2022 hasta el 20 de abril del 2023, es decir que solo se tiene registro de 7 meses



In [ ]:
# Obtiene los valores únicos de la columna 'DAYOFWEEK'
valores_unicos_dayofweek = bogota.select("MONTH").distinct().rdd.flatMap(lambda x: x).collect()

# Imprime los valores únicos de 'DAYOFWEEK'
print("Valores únicos de MONTH:", valores_unicos_dayofweek)

Valores únicos de MONTH: [1, 10, 3, 12, 11, 2, 4]


In [ ]:
# Obtiene los valores únicos de la columna 'DAYOFWEEK'
valores_unicos_dayofweek = bogota.select("DAYOFWEEK").distinct().rdd.flatMap(lambda x: x).collect()

# Imprime los valores únicos de 'DAYOFWEEK'
print("Valores únicos de DAYOFWEEK:", valores_unicos_dayofweek)

Valores únicos de DAYOFWEEK: [7, 6, 5, 1, 3, 2, 4]


In [ ]:
# Agrega una nueva columna 'YEAR' basada en la columna 'COMPLETEDATE'
bogota = bogota.withColumn('YEAR', year('COMPLETEDATE'))

# Muestra las primeras filas del DataFrame resultante
bogota.show()

+--------+-----+-----------+------------------+------------------+------------------+------------------+----------+-------+-------------------+----------+---------+----+-----+----+
|      ID| CITY|       USER|       LATITUDEORI|      LATITUDEDEST|      LONGITUDEORI|     LONGITUDEDEST|    STATUS|CHANNEL|       COMPLETEDATE|      DATE|DAYOFWEEK|HOUR|MONTH|YEAR|
+--------+-----+-----------+------------------+------------------+------------------+------------------+----------+-------+-------------------+----------+---------+----+-----+----+
|80206465|11001|  8168434.0| 4.646834566745485| 4.669336700000088|-74.10161945968866|-74.11134894799994| CANCELADO|    APP|2022-10-01 00:00:00|2022-10-01|        6|   0|   10|2022|
|80206467|11001|1.0186684E7| 4.607888513534026|           4.54708|-74.07884381711483|         -74.14815|ABANDONADO|    APP|2022-10-01 00:00:00|2022-10-01|        6|   0|   10|2022|
|80206470|11001|  8318357.0| 4.603995829146333| 4.641824628000053|-74.12916909903288| -74.15346

In [ ]:
# Agrega una nueva columna 'COMPLETEHOUR' basada en la columna 'COMPLETEDATE'
bogota = bogota.withColumn('COMPLETEHOUR', date_format('COMPLETEDATE', 'HH:mm:ss'))

# Muestra las primeras filas del DataFrame resultante
bogota.show()

+--------+-----+-----------+------------------+------------------+------------------+------------------+----------+-------+-------------------+----------+---------+----+-----+----+------------+
|      ID| CITY|       USER|       LATITUDEORI|      LATITUDEDEST|      LONGITUDEORI|     LONGITUDEDEST|    STATUS|CHANNEL|       COMPLETEDATE|      DATE|DAYOFWEEK|HOUR|MONTH|YEAR|COMPLETEHOUR|
+--------+-----+-----------+------------------+------------------+------------------+------------------+----------+-------+-------------------+----------+---------+----+-----+----+------------+
|80206465|11001|  8168434.0| 4.646834566745485| 4.669336700000088|-74.10161945968866|-74.11134894799994| CANCELADO|    APP|2022-10-01 00:00:00|2022-10-01|        6|   0|   10|2022|    00:00:00|
|80206467|11001|1.0186684E7| 4.607888513534026|           4.54708|-74.07884381711483|         -74.14815|ABANDONADO|    APP|2022-10-01 00:00:00|2022-10-01|        6|   0|   10|2022|    00:00:00|
|80206470|11001|  8318357.0| 4

#3.Exploración del dataset

## Estadística desciptiva

In [ ]:
# Selecciona las columnas de interés
selected_columns = ["STATUS", "CHANNEL", "DATE", "YEAR", "MONTH", "DAYOFWEEK", "HOUR"]

# Filtra el DataFrame para seleccionar solo las columnas de interés
bogota_fechas = bogota.select(selected_columns)

# Calcula estadísticas descriptivas para las columnas numéricas
numeric_stats = bogota_fechas.describe(["YEAR", "MONTH", "DAYOFWEEK", "HOUR"])

# Calcula la frecuencia de valores únicos para las columnas categóricas
status_counts = bogota_fechas.groupBy("STATUS").count()
channel_counts = bogota_fechas.groupBy("CHANNEL").count()

# Muestra las estadísticas descriptivas y la frecuencia de valores únicos
numeric_stats.show()
status_counts.show()
channel_counts.show()

+-------+------------------+------------------+------------------+------------------+
|summary|              YEAR|             MONTH|         DAYOFWEEK|              HOUR|
+-------+------------------+------------------+------------------+------------------+
|  count|          12005856|          12005856|          12005856|          12005856|
|   mean|2022.4133009757904|7.4684538944994845|3.9228429859561866|13.304646665760442|
| stddev| 0.492425932897974| 4.437203478499236|1.8373463910108858| 7.659670083027695|
|    min|              2022|                 1|                 1|                 0|
|    max|              2023|                12|                 7|                23|
+-------+------------------+------------------+------------------+------------------+

+----------+-------+
|    STATUS|  count|
+----------+-------+
| CANCELADO|1355486|
|ABANDONADO|7398640|
|  CUMPLIDO|3251730|
+----------+-------+

+------------+--------+
|     CHANNEL|   count|
+------------+--------+
|    

In [ ]:
#Tabla de Contingencia entre STATUS y CHANNEL
contingency_table = bogota_fechas.crosstab("STATUS", "CHANNEL")
contingency_table.show()

+--------------+-------+------------+------+---------+
|STATUS_CHANNEL|    APP|CHAT_BOT_659|   IVR|RECEPCION|
+--------------+-------+------------+------+---------+
|     CANCELADO|1354439|           0|     0|     1047|
|    ABANDONADO|7393064|           0|     0|     5576|
|      CUMPLIDO|2607660|      190205|371578|    82287|
+--------------+-------+------------+------+---------+



In [ ]:
# Muestra el nuevo DataFrame
bogota_fechas.show()

+----------+-------+----------+----+-----+---------+----+
|    STATUS|CHANNEL|      DATE|YEAR|MONTH|DAYOFWEEK|HOUR|
+----------+-------+----------+----+-----+---------+----+
| CANCELADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|  CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|ABANDONADO|    APP|2022-10-01|2022|   10|        6|   0|
|  CUMPLIDO|  

ABANDONADO: El mes que tuvo mayor cantidad de abandonos fue el mes 12(Diciembre) 2022 == 2.1 millones ABANDONADO: El mes que tuvo menor cantidad de abandonos fue el mes 4(abril) 2023 == 263.6 mil

CANCELADO: El mes que tuvo mayor cantidad de cancelaciones fue el mes 12(Diciembre) 2022 == 268.3 mil CANCELADO: El mes que tuvo menor cantidad de cancelaciones fue el mes 4(abril) 2023 == 89.6 mil

CUMPLIDO: El mes que tuvo mayor cantidad de viajes completados fue el mes 10(Octubre) 2022 == 541 mil CUMPLIDO: El mes que tuvo menor cantidad de viajes completados fue el mes 4(abril) 2023 == 252.6 mil

In [ ]:
bogota_fechas.count()

12005856

### Viajes cumplidos por app

In [ ]:
cumplido_app_df = bogota_fechas.filter((bogota_fechas['STATUS'] == 'CUMPLIDO') & (bogota_fechas['CHANNEL'] == 'APP'))

In [ ]:
# Obtiene los valores únicos de la columna 'DAYOFWEEK'
day_of_week_values = cumplido_app_df.select("DAYOFWEEK").distinct().orderBy("DAYOFWEEK")

# Convierte los resultados en una lista de Python
day_of_week_list = [row.DAYOFWEEK for row in day_of_week_values.collect()]

# Muestra la lista de DAYOFWEEK
print(day_of_week_list)

[1, 2, 3, 4, 5, 6, 7]


In [ ]:
cumplido_app_df.count()

2607660

In [ ]:
cumplido_app_df.show()

+--------+-------+----------+----+-----+---------+----+
|  STATUS|CHANNEL|      DATE|YEAR|MONTH|DAYOFWEEK|HOUR|
+--------+-------+----------+----+-----+---------+----+
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        6|   0|
|CUMPLIDO|    APP|2022-10-01|2022|   10|        

#### Cantidad de viajes cumplidos al mes

In [ ]:
# Agrupa los viajes cumplidos por fecha y cuenta la frecuencia
frecuencia_por_fecha = cumplido_app_df.groupBy("DATE").count().orderBy("DATE")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas = frecuencia_por_fecha.toPandas()

# Crea el histograma con Plotly
fig = px.bar(data_pandas, x="DATE", y="count", title="Histograma de la cantidad de Viajes Cumplidos por mes (APP)")

# Muestra el histograma
fig.show()

A partir de la gráfica, se evidencia que el 28 de Febrero del 2023, due el día con más viajes cumplidos por App. Por otro lado, el 7 de Abril del 2023, se presentaron menos en general con un número de 4177

In [ ]:
# Crea la línea de tiempo con Plotly
fig = px.line(data_pandas, x="DATE", y="count", markers=True, title='Línea de Tiempo de la cantidad de Viajes Cumplidos por mes')

# Personaliza las etiquetas del eje x para una mejor visualización
fig.update_xaxes(title_text='Fecha (Año-Mes)', tickangle=45, showgrid=True)

# Personaliza las etiquetas del eje y
fig.update_yaxes(title_text='Viajes', showgrid=True)

# Muestra la línea de tiempo
fig.show()

#### Cantidad de viajes cumplidos semanal

In [ ]:
# Filtra los registros para el día "Lunes" (DAYOFWEEK = 1)
lunes_data = cumplido_app_df.filter(col("DAYOFWEEK") == 1)

# Cuenta cuántos registros hay para el día "Lunes"
conteo_lunes = lunes_data.count()

# Imprime el conteo
print("Número de registros para el día Lunes:", conteo_lunes)

Número de registros para el día Lunes: 338542


In [ ]:
# Agrupa los viajes cumplidos por el día de la semana y cuenta la frecuencia
frecuencia_por_dia_semana = cumplido_app_df.groupBy("DAYOFWEEK").count().orderBy("DAYOFWEEK")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas = frecuencia_por_dia_semana.toPandas()

# Mapea los números de día de la semana a sus nombres correspondientes
dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
data_pandas['DAYOFWEEK'] = data_pandas['DAYOFWEEK'].map({i: dia for i, dia in enumerate(dias_semana, start=1)})

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig = px.bar(data_pandas, x="DAYOFWEEK", y="count", title="Histograma de la cantidad de viajes Cumplidos por Día de la Semana (APP)")
fig.update_xaxes(title_text="Día de la Semana", categoryorder='array', categoryarray=dias_semana)

# Muestra el histograma
fig.show()

En general, el día viernes fue el más concurrido de viajes cumplidos por medio de una App con 411.740 recorridos. Por otro lado, el día con menos concurrencia fue el domingo con 279.752

In [ ]:
# Crea una línea de tiempo con puntos en cada día de la semana
fig_timeline = px.line(data_pandas, x="DAYOFWEEK", y="count", title="Línea de Tiempo de la cantidad de viajes Cumplidos por Día de la Semana (APP)")

# Personaliza los ejes
fig_timeline.update_xaxes(title_text="Día de la Semana")
fig_timeline.update_yaxes(title_text="Viajes")

# Agrega puntos a la línea de tiempo
fig_timeline.update_traces(mode="markers+lines", marker=dict(size=8))

# Muestra la línea de tiempo
fig_timeline.show()

#### Cantidad de Viajes cumplidos por hora

In [ ]:
# Agrupa los viajes cumplidos por la hora completa del día y cuenta la frecuencia
frecuencia_por_hora_completa = cumplido_app_df.groupBy("HOUR").count().orderBy("HOUR")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_hour_completa = frecuencia_por_hora_completa.toPandas()

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig_hour_completa = px.bar(data_pandas_hour_completa, x="HOUR", y="count", title="Histograma de cantidad de Viajes Cumplidos por Hora Completa del Día (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_hour_completa.update_xaxes(title_text="Hora Completa del Día", tickmode='linear', tick0=0, dtick=1)

# Muestra el histograma
fig_hour_completa.show()

Se evidencia que la hora más concurrida de viajes es a las 11:00 a.m con 139.486 recorridos. Por otro lado, la hora con menos frecuencia fueron las 7:00 am con 36.739.

In [ ]:
# Crea la serie de tiempo con Plotly
fig_time_series = px.line(data_pandas_hour_completa, x="HOUR", y="count", title="Serie de Tiempo de la cantidad de Viajes Cumplidos por Hora Completa del Día (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_time_series.update_xaxes(title_text="Hora Completa del Día", tickmode='array', tickvals=data_pandas_hour_completa['HOUR'], ticktext=data_pandas_hour_completa['HOUR'])

# Agrega puntos para identificar mejor los datos
fig_time_series.update_traces(mode="markers+lines", marker=dict(size=4))

# Muestra la serie de tiempo
fig_time_series.show()

In [ ]:
# Filtra los datos para obtener solo los registros desde noviembre de 2022 hasta abril de 2023
octubre_2022_df_cumplido = bogota_fechas.filter(
    (bogota_fechas['YEAR'] == '2022') & (bogota_fechas['MONTH'] == '10') &
    (bogota_fechas['STATUS'] == 'CUMPLIDO') & (bogota_fechas['CHANNEL'] == 'APP')
)

noviembre_2022_df_cumplido = bogota_fechas.filter(
    (bogota_fechas['YEAR'] == '2022') & (bogota_fechas['MONTH'] == '11') &
    (bogota_fechas['STATUS'] == 'CUMPLIDO') & (bogota_fechas['CHANNEL'] == 'APP')
)

diciembre_2022_df_cumplido = bogota_fechas.filter(
    (bogota_fechas['YEAR'] == '2022') & (bogota_fechas['MONTH'] == '12') &
    (bogota_fechas['STATUS'] == 'CUMPLIDO') & (bogota_fechas['CHANNEL'] == 'APP')
)

enero_2023_df_cumplido = bogota_fechas.filter(
    (bogota_fechas['YEAR'] == '2023') & (bogota_fechas['MONTH'] == '01') &
    (bogota_fechas['STATUS'] == 'CUMPLIDO') & (bogota_fechas['CHANNEL'] == 'APP')
)

febrero_2023_df_cumplido = bogota_fechas.filter(
    (bogota_fechas['YEAR'] == '2023') & (bogota_fechas['MONTH'] == '02') &
    (bogota_fechas['STATUS'] == 'CUMPLIDO') & (bogota_fechas['CHANNEL'] == 'APP')
)

marzo_2023_df_cumplido = bogota_fechas.filter(
    (bogota_fechas['YEAR'] == '2023') & (bogota_fechas['MONTH'] == '03') &
    (bogota_fechas['STATUS'] == 'CUMPLIDO') & (bogota_fechas['CHANNEL'] == 'APP')
)

abril_2023_df_cumplido = bogota_fechas.filter(
    (bogota_fechas['YEAR'] == '2023') & (bogota_fechas['MONTH'] == '04') &
    (bogota_fechas['STATUS'] == 'CUMPLIDO') & (bogota_fechas['CHANNEL'] == 'APP')
)


In [ ]:
# Cuenta los registros en cada DataFrame
count_octubre_2022 = octubre_2022_df_cumplido.count()
count_noviembre_2022 = noviembre_2022_df_cumplido.count()
count_diciembre_2022 = diciembre_2022_df_cumplido.count()
count_enero_2023 = enero_2023_df_cumplido.count()
count_febrero_2023 = febrero_2023_df_cumplido.count()
count_marzo_2023 = marzo_2023_df_cumplido.count()
count_abril_2023 = abril_2023_df_cumplido.count()

# Imprime los resultados
print("Registros en noviembre de 2022 de viajes cumplidos por app:", count_noviembre_2022)
print("Registros en diciembre de 2022 de viajes cumplidos por app:", count_diciembre_2022)
print("Registros en enero de 2023 de viajes cumplidos por app:", count_enero_2023)
print("Registros en febrero de 2023 de viajes cumplidos por app:", count_febrero_2023)
print("Registros en marzo de 2023 de viajes cumplidos por app:", count_marzo_2023)
print("Registros en abril de 2023 de viajes cumplidos por app:", count_abril_2023)


Registros en noviembre de 2022 de viajes cumplidos por app: 406233
Registros en diciembre de 2022 de viajes cumplidos por app: 421020
Registros en enero de 2023 de viajes cumplidos por app: 350994
Registros en febrero de 2023 de viajes cumplidos por app: 388288
Registros en marzo de 2023 de viajes cumplidos por app: 413496
Registros en abril de 2023 de viajes cumplidos por app: 195413


Estos resultados representan la cantidad de viajes cumplidos por la aplicación (APP) en cada mes específico, desde noviembre de 2022 hasta abril de 2023. Aquí hay una interpretación de estos datos:

* Noviembre de 2022 - 406,233 registros: Este mes muestra una cantidad significativa de viajes cumplidos a través de la aplicación. Puede haber varias razones para esto, como un aumento en la adopción de la aplicación por parte de los usuarios o eventos estacionales que generaron una mayor demanda de viajes.

* Diciembre de 2022 - 421,020 registros: En diciembre, la cantidad de viajes cumplidos por la aplicación es ligeramente superior a la de noviembre. Esto podría deberse a eventos festivos y vacaciones, ya que las personas tienden a viajar más durante la temporada navideña.

* Enero de 2023 - 350,994 registros: Enero muestra una disminución en la cantidad de viajes cumplidos en comparación con diciembre. Esta disminución podría deberse a la finalización de la temporada navideña y a que las personas vuelven a la rutina después de las festividades.

* Febrero de 2023 - 388,288 registros: En febrero, la cantidad de viajes cumplidos vuelve a aumentar en comparación con enero. Las posibles razones podrían incluir eventos especiales, promociones u otros factores que impulsaron la demanda de viajes.

* Marzo de 2023 - 413,496 registros: Marzo muestra un aumento adicional en la cantidad de viajes cumplidos. Este aumento puede deberse a factores como el cambio de estaciones o la promoción de la aplicación.

* Abril de 2023 - 195,413 registros: En abril, la cantidad de viajes cumplidos por la aplicación disminuye drásticamente en comparación con los meses anteriores. Puede haber varios factores que contribuyan a esta disminución, como la estacionalidad o cambios en la demanda de viajes.

Estos resultados son útiles para comprender las fluctuaciones en la demanda de viajes a lo largo de los meses y pueden ayudar en la planificación y toma de decisiones relacionadas con los servicios de transporte y la asignación de recursos. También es importante considerar factores externos, como eventos, promociones o tendencias estacionales, que pueden influir en la cantidad de viajes cumplidos.

##### Viajes cumplidos en Octubre del 2022


In [ ]:
import datetime

###### Cantidad de viajes por día en Octubre del 2022

In [ ]:
# Agrupa los viajes cumplidos por fecha y cuenta la frecuencia
frecuencia_por_fecha = octubre_2022_df_cumplido.groupBy("DATE").count().orderBy("DATE")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_dias_octubre = frecuencia_por_fecha.toPandas()

# Crea el histograma con Plotly y establece las etiquetas en el eje x
fig = px.bar(data_pandas_dias_octubre, x="DATE", y="count", title="Histograma de la cantidad de Viajes Cumplidos por día en Octubre de 2022 (APP)")

# Muestra el histograma
fig.show()

In [ ]:
# Genera una secuencia de fechas para el eje x
fechas = pd.date_range(start="2022-10-01", end="2022-10-31")

# Crea la línea de tiempo con Plotly
fig = px.line(data_pandas_dias_octubre, x=fechas, y="count", title="Línea de Tiempo de la cantidad de Viajes Cumplidos en Octubre de 2022 (APP)")

# Personaliza el formato de las etiquetas del eje x
fig.update_xaxes(
    tickvals=fechas,
    ticktext=[fecha.strftime('%Y-%m-%d') for fecha in fechas],
    tickangle=45,
)

# Muestra la línea de tiempo
fig.show()

###### Cantidad de viajes cumplidos por día de la semana en Octubre del 2022

In [ ]:
# Agrupa los viajes cumplidos por el día de la semana y cuenta la frecuencia
frecuencia_por_dia_semana = octubre_2022_df_cumplido.groupBy("DAYOFWEEK").count().orderBy("DAYOFWEEK")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_semanal_octubre = frecuencia_por_dia_semana.toPandas()

# Mapea los números de día de la semana a sus nombres correspondientes
dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
data_pandas_semanal_octubre['DAYOFWEEK'] = data_pandas_semanal_octubre['DAYOFWEEK'].map({i: dia for i, dia in enumerate(dias_semana, start=1)})

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig = px.bar(data_pandas_semanal_octubre, x="DAYOFWEEK", y="count", title="Histograma de la cantidad de viajes Cumplidos en Octubre de 2022 (APP)")
fig.update_xaxes(title_text="Día de la Semana", categoryorder='array', categoryarray=dias_semana)

# Muestra el histograma
fig.show()

In [ ]:
# Crea una línea de tiempo con puntos en cada día de la semana
fig_timeline = px.line(data_pandas_semanal_octubre, x="DAYOFWEEK", y="count", title="Línea de Tiempo de la cantidad de viajes Cumplidos por Día de la Semana (APP)")

# Personaliza los ejes
fig_timeline.update_xaxes(title_text="Día de la Semana")
fig_timeline.update_yaxes(title_text="Viajes")

# Agrega puntos a la línea de tiempo
fig_timeline.update_traces(mode="markers+lines", marker=dict(size=8))

# Muestra la línea de tiempo
fig_timeline.show()

###### Cantidad de viajes cumplidos por hora en Octubre del 2022

In [ ]:
# Agrupa los viajes cumplidos por la hora completa del día y cuenta la frecuencia
frecuencia_por_hora_completa = octubre_2022_df_cumplido.groupBy("HOUR").count().orderBy("HOUR")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_horas_octubre = frecuencia_por_hora_completa.toPandas()

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig_hour_completa = px.bar(data_pandas_horas_octubre, x="HOUR", y="count", title="Histograma de cantidad de Viajes Cumplidos por en octubre de 2022 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_hour_completa.update_xaxes(title_text="Hora Completa del Día", tickmode='linear', tick0=0, dtick=1)

# Muestra el histograma
fig_hour_completa.show()

In [ ]:
# Crea la serie de tiempo con Plotly
fig_time_series = px.line(data_pandas_horas_octubre, x="HOUR", y="count", title="Serie de Tiempo de la cantidad de Viajes Cumplidos por hora en octubre de 2022 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_time_series.update_xaxes(title_text="Hora Completa del Día", tickmode='array', tickvals=data_pandas_horas_octubre['HOUR'], ticktext=data_pandas_horas_octubre['HOUR'])

# Agrega puntos para identificar mejor los datos
fig_time_series.update_traces(mode="markers+lines", marker=dict(size=4))

# Muestra la serie de tiempo
fig_time_series.show()

##### Viajes en Noviembre del 2022


###### Cantidad de viajes cumplidos por día en Noviembre del 2022

In [ ]:
# Agrupa los viajes cumplidos por fecha y cuenta la frecuencia
frecuencia_por_fecha = noviembre_2022_df_cumplido .groupBy("DATE").count().orderBy("DATE")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_dias_noviembre = frecuencia_por_fecha.toPandas()

# Crea el histograma con Plotly y establece las etiquetas en el eje x
fig = px.bar(data_pandas_dias_noviembre, x="DATE", y="count", title="Histograma de la cantidad de Viajes Cumplidos por día en noviembre de 2022 (APP)")

# Muestra el histograma
fig.show()

In [ ]:
# Genera una secuencia de fechas para el eje x
fechas = pd.date_range(start="2022-11-01", end="2022-11-30")

# Crea la línea de tiempo con Plotly
fig = px.line(data_pandas_dias_noviembre, x=fechas, y="count", title="Línea de Tiempo de la cantidad de Viajes Cumplidos en noviembre de 2022 (APP)")

# Personaliza el formato de las etiquetas del eje x
fig.update_xaxes(
    tickvals=fechas,
    ticktext=[fecha.strftime('%Y-%m-%d') for fecha in fechas],
    tickangle=45,
)

# Muestra la línea de tiempo
fig.show()

###### Cantidad de viajes semanales en Noviembre del 2022

In [ ]:
# Agrupa los viajes cumplidos por el día de la semana y cuenta la frecuencia
frecuencia_por_dia_semana = noviembre_2022_df_cumplido.groupBy("DAYOFWEEK").count().orderBy("DAYOFWEEK")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_semanal_noviembre = frecuencia_por_dia_semana.toPandas()

# Mapea los números de día de la semana a sus nombres correspondientes
dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
data_pandas_semanal_noviembre['DAYOFWEEK'] = data_pandas_semanal_noviembre['DAYOFWEEK'].map({i: dia for i, dia in enumerate(dias_semana, start=1)})

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig = px.bar(data_pandas_semanal_noviembre, x="DAYOFWEEK", y="count", title="Histograma de la cantidad de viajes Cumplidos en noviembre de 2022 (APP)")
fig.update_xaxes(title_text="Día de la Semana", categoryorder='array', categoryarray=dias_semana)

# Muestra el histograma
fig.show()

In [ ]:
# Crea una línea de tiempo con puntos en cada día de la semana
fig_timeline = px.line(data_pandas_semanal_noviembre, x="DAYOFWEEK", y="count", title="Línea de Tiempo de la cantidad de viajes Cumplidos por Día de la Semana (APP)")

# Personaliza los ejes
fig_timeline.update_xaxes(title_text="Día de la Semana")
fig_timeline.update_yaxes(title_text="Viajes")

# Agrega puntos a la línea de tiempo
fig_timeline.update_traces(mode="markers+lines", marker=dict(size=8))

# Muestra la línea de tiempo
fig_timeline.show()

###### Cantidad de viajes cumplidos por hora en Noviembre del 2022

In [ ]:
# Agrupa los viajes cumplidos por la hora completa del día y cuenta la frecuencia
frecuencia_por_hora_completa = noviembre_2022_df_cumplido.groupBy("HOUR").count().orderBy("HOUR")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_horas_noviembre = frecuencia_por_hora_completa.toPandas()

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig_hour_completa = px.bar(data_pandas_horas_noviembre, x="HOUR", y="count", title="Histograma de cantidad de Viajes Cumplidos por en noviembre de 2022 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_hour_completa.update_xaxes(title_text="Hora Completa del Día", tickmode='linear', tick0=0, dtick=1)

# Muestra el histograma
fig_hour_completa.show()

In [ ]:
# Crea la serie de tiempo con Plotly
fig_time_series = px.line(data_pandas_horas_noviembre, x="HOUR", y="count", title="Serie de Tiempo de la cantidad de Viajes Cumplidos por hora en noviembre de 2022 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_time_series.update_xaxes(title_text="Hora Completa del Día", tickmode='array', tickvals=data_pandas_horas_noviembre['HOUR'], ticktext=data_pandas_horas_noviembre['HOUR'])

# Agrega puntos para identificar mejor los datos
fig_time_series.update_traces(mode="markers+lines", marker=dict(size=4))

# Muestra la serie de tiempo
fig_time_series.show()

##### Viajes en Diciembre del 2022


###### Cantidad de viajes cumplidos por día en Diciembre del 2022

In [ ]:
# Agrupa los viajes cumplidos por fecha y cuenta la frecuencia
frecuencia_por_fecha = diciembre_2022_df_cumplido .groupBy("DATE").count().orderBy("DATE")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_dias_diciembre = frecuencia_por_fecha.toPandas()

# Crea el histograma con Plotly y establece las etiquetas en el eje x
fig = px.bar(data_pandas_dias_diciembre, x="DATE", y="count", title="Histograma de la cantidad de Viajes Cumplidos por día en diciembre de 2022 (APP)")

# Muestra el histograma
fig.show()

In [ ]:
# Genera una secuencia de fechas para el eje x
fechas = pd.date_range(start="2022-12-01", end="2022-12-31")

# Crea la línea de tiempo con Plotly
fig = px.line(data_pandas_dias_diciembre, x=fechas, y="count", title="Línea de Tiempo de la cantidad de Viajes Cumplidos en diciembre de 2022 (APP)")

# Personaliza el formato de las etiquetas del eje x
fig.update_xaxes(
    tickvals=fechas,
    ticktext=[fecha.strftime('%Y-%m-%d') for fecha in fechas],
    tickangle=45,
)

# Muestra la línea de tiempo
fig.show()

###### Cantidad de viajes semanales en Diciembre del 2022

In [ ]:
# Agrupa los viajes cumplidos por el día de la semana y cuenta la frecuencia
frecuencia_por_dia_semana = diciembre_2022_df_cumplido.groupBy("DAYOFWEEK").count().orderBy("DAYOFWEEK")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_semanal_diciembre = frecuencia_por_dia_semana.toPandas()

# Mapea los números de día de la semana a sus nombres correspondientes
dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
data_pandas_semanal_diciembre['DAYOFWEEK'] = data_pandas_semanal_diciembre['DAYOFWEEK'].map({i: dia for i, dia in enumerate(dias_semana, start=1)})

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig = px.bar(data_pandas_semanal_diciembre, x="DAYOFWEEK", y="count", title="Histograma de la cantidad de viajes Cumplidos en diciembre de 2022 (APP)")
fig.update_xaxes(title_text="Día de la Semana", categoryorder='array', categoryarray=dias_semana)

# Muestra el histograma
fig.show()

In [ ]:
# Crea una línea de tiempo con puntos en cada día de la semana
fig_timeline = px.line(data_pandas_semanal_diciembre, x="DAYOFWEEK", y="count", title="Línea de Tiempo de la cantidad de viajes Cumplidos por Día de la Semana (APP)")

# Personaliza los ejes
fig_timeline.update_xaxes(title_text="Día de la Semana")
fig_timeline.update_yaxes(title_text="Viajes")

# Agrega puntos a la línea de tiempo
fig_timeline.update_traces(mode="markers+lines", marker=dict(size=8))

# Muestra la línea de tiempo
fig_timeline.show()

###### Cantidad de viajes cumplidos por hora en Diciembre del 2022

In [ ]:
# Agrupa los viajes cumplidos por la hora completa del día y cuenta la frecuencia
frecuencia_por_hora_completa = diciembre_2022_df_cumplido .groupBy("HOUR").count().orderBy("HOUR")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_horas_diciembre = frecuencia_por_hora_completa.toPandas()

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig_hour_completa = px.bar(data_pandas_horas_diciembre, x="HOUR", y="count", title="Histograma de cantidad de Viajes Cumplidos por en diciembre de 2022 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_hour_completa.update_xaxes(title_text="Hora Completa del Día", tickmode='linear', tick0=0, dtick=1)

# Muestra el histograma
fig_hour_completa.show()

In [ ]:
# Crea la serie de tiempo con Plotly
fig_time_series = px.line(data_pandas_horas_diciembre, x="HOUR", y="count", title="Serie de Tiempo de la cantidad de Viajes Cumplidos por hora en diciembre de 2022 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_time_series.update_xaxes(title_text="Hora Completa del Día", tickmode='array', tickvals=data_pandas_horas_diciembre['HOUR'], ticktext=data_pandas_horas_diciembre['HOUR'])

# Agrega puntos para identificar mejor los datos
fig_time_series.update_traces(mode="markers+lines", marker=dict(size=4))

# Muestra la serie de tiempo
fig_time_series.show()

##### Viajes en Enero del 2022


###### Cantidad de viajes cumplidos por día en Enero del 2023

In [ ]:
# Agrupa los viajes cumplidos por fecha y cuenta la frecuencia
frecuencia_por_fecha = enero_2023_df_cumplido .groupBy("DATE").count().orderBy("DATE")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_dias_enero = frecuencia_por_fecha.toPandas()

# Crea el histograma con Plotly y establece las etiquetas en el eje x
fig = px.bar(data_pandas_dias_enero, x="DATE", y="count", title="Histograma de la cantidad de Viajes Cumplidos por día en enero de 2023 (APP)")

# Muestra el histograma
fig.show()

In [ ]:
# Genera una secuencia de fechas para el eje x
fechas = pd.date_range(start="2023-1-01", end="2023-1-31")

# Crea la línea de tiempo con Plotly
fig = px.line(data_pandas_dias_enero, x=fechas, y="count", title="Línea de Tiempo de la cantidad de Viajes Cumplidos en enero de 2023 (APP)")

# Personaliza el formato de las etiquetas del eje x
fig.update_xaxes(
    tickvals=fechas,
    ticktext=[fecha.strftime('%Y-%m-%d') for fecha in fechas],
    tickangle=45,
)

# Muestra la línea de tiempo
fig.show()

###### Cantidad de viajes semanales en Enero del 2022

In [ ]:
# Agrupa los viajes cumplidos por el día de la semana y cuenta la frecuencia
frecuencia_por_dia_semana = enero_2023_df_cumplido.groupBy("DAYOFWEEK").count().orderBy("DAYOFWEEK")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_semanal_enero = frecuencia_por_dia_semana.toPandas()

# Mapea los números de día de la semana a sus nombres correspondientes
dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
data_pandas_semanal_enero['DAYOFWEEK'] = data_pandas_semanal_enero['DAYOFWEEK'].map({i: dia for i, dia in enumerate(dias_semana, start=1)})

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig = px.bar(data_pandas_semanal_enero, x="DAYOFWEEK", y="count", title="Histograma de la cantidad de viajes Cumplidos en enero de 2023 (APP)")
fig.update_xaxes(title_text="Día de la Semana", categoryorder='array', categoryarray=dias_semana)

# Muestra el histograma
fig.show()

In [ ]:
# Crea una línea de tiempo con puntos en cada día de la semana
fig_timeline = px.line(data_pandas_semanal_enero, x="DAYOFWEEK", y="count", title="Línea de Tiempo de la cantidad de viajes Cumplidos por Día de la Semana (APP)")

# Personaliza los ejes
fig_timeline.update_xaxes(title_text="Día de la Semana")
fig_timeline.update_yaxes(title_text="Viajes")

# Agrega puntos a la línea de tiempo
fig_timeline.update_traces(mode="markers+lines", marker=dict(size=8))

# Muestra la línea de tiempo
fig_timeline.show()

###### Cantidad de viajes cumplidos por hora en Enero del 2023

In [ ]:
# Agrupa los viajes cumplidos por la hora completa del día y cuenta la frecuencia
frecuencia_por_hora_completa = enero_2023_df_cumplido.groupBy("HOUR").count().orderBy("HOUR")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_horas_enero = frecuencia_por_hora_completa.toPandas()

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig_hour_completa = px.bar(data_pandas_horas_enero, x="HOUR", y="count", title="Histograma de cantidad de Viajes Cumplidos por en enero del 2023 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_hour_completa.update_xaxes(title_text="Hora Completa del Día", tickmode='linear', tick0=0, dtick=1)

# Muestra el histograma
fig_hour_completa.show()

In [ ]:
# Crea la serie de tiempo con Plotly
fig_time_series = px.line(data_pandas_horas_enero, x="HOUR", y="count", title="Serie de Tiempo de la cantidad de Viajes Cumplidos por hora en enero de 2023 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_time_series.update_xaxes(title_text="Hora Completa del Día", tickmode='array', tickvals=data_pandas_horas_enero['HOUR'], ticktext=data_pandas_horas_enero['HOUR'])

# Agrega puntos para identificar mejor los datos
fig_time_series.update_traces(mode="markers+lines", marker=dict(size=4))

# Muestra la serie de tiempo
fig_time_series.show()

##### Viajes en Febrero del 2022


###### Cantidad de viajes cumplidos por día en Febrero del 2023

In [ ]:
# Agrupa los viajes cumplidos por fecha y cuenta la frecuencia
frecuencia_por_fecha = febrero_2023_df_cumplido.groupBy("DATE").count().orderBy("DATE")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_dias_febrero = frecuencia_por_fecha.toPandas()

# Crea el histograma con Plotly y establece las etiquetas en el eje x
fig = px.bar(data_pandas_dias_febrero, x="DATE", y="count", title="Histograma de la cantidad de Viajes Cumplidos por día en febrero de 2023 (APP)")

# Muestra el histograma
fig.show()

In [ ]:
# Genera una secuencia de fechas para el eje x
fechas = pd.date_range(start="2023-2-01", end="2023-2-28")

# Crea la línea de tiempo con Plotly
fig = px.line(data_pandas_dias_febrero, x=fechas, y="count", title="Línea de Tiempo de la cantidad de Viajes Cumplidos en febero de 2023 (APP)")

# Personaliza el formato de las etiquetas del eje x
fig.update_xaxes(
    tickvals=fechas,
    ticktext=[fecha.strftime('%Y-%m-%d') for fecha in fechas],
    tickangle=45,
)

# Muestra la línea de tiempo
fig.show()

###### Cantidad de viajes semanales en Febrero del 2022

In [ ]:
# Agrupa los viajes cumplidos por el día de la semana y cuenta la frecuencia
frecuencia_por_dia_semana = febrero_2023_df_cumplido.groupBy("DAYOFWEEK").count().orderBy("DAYOFWEEK")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_semanal_febrero = frecuencia_por_dia_semana.toPandas()

# Mapea los números de día de la semana a sus nombres correspondientes
dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
data_pandas_semanal_febrero['DAYOFWEEK'] = data_pandas_semanal_febrero['DAYOFWEEK'].map({i: dia for i, dia in enumerate(dias_semana, start=1)})

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig = px.bar(data_pandas_semanal_febrero, x="DAYOFWEEK", y="count", title="Histograma de la cantidad de viajes Cumplidos en febrero de 2023 (APP)")
fig.update_xaxes(title_text="Día de la Semana", categoryorder='array', categoryarray=dias_semana)

# Muestra el histograma
fig.show()

In [ ]:
# Crea una línea de tiempo con puntos en cada día de la semana
fig_timeline = px.line(data_pandas_semanal_febrero, x="DAYOFWEEK", y="count", title="Línea de Tiempo de la cantidad de viajes Cumplidos por Día de la Semana (APP)")

# Personaliza los ejes
fig_timeline.update_xaxes(title_text="Día de la Semana")
fig_timeline.update_yaxes(title_text="Viajes")

# Agrega puntos a la línea de tiempo
fig_timeline.update_traces(mode="markers+lines", marker=dict(size=8))

# Muestra la línea de tiempo
fig_timeline.show()

###### Cantidad de viajes cumplidos por hora en Febrero del 2022

In [ ]:
# Agrupa los viajes cumplidos por la hora completa del día y cuenta la frecuencia
frecuencia_por_hora_completa = febrero_2023_df_cumplido.groupBy("HOUR").count().orderBy("HOUR")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_horas_febrero = frecuencia_por_hora_completa.toPandas()

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig_hour_completa = px.bar(data_pandas_horas_febrero, x="HOUR", y="count", title="Histograma de cantidad de Viajes Cumplidos por en febrero de 2023 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_hour_completa.update_xaxes(title_text="Hora Completa del Día", tickmode='linear', tick0=0, dtick=1)

# Muestra el histograma
fig_hour_completa.show()

In [ ]:
# Crea la serie de tiempo con Plotly
fig_time_series = px.line(data_pandas_horas_febrero, x="HOUR", y="count", title="Serie de Tiempo de la cantidad de Viajes Cumplidos por hora en febrero de 2023 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_time_series.update_xaxes(title_text="Hora Completa del Día", tickmode='array', tickvals=data_pandas_horas_febrero['HOUR'], ticktext=data_pandas_horas_febrero['HOUR'])

# Agrega puntos para identificar mejor los datos
fig_time_series.update_traces(mode="markers+lines", marker=dict(size=4))

# Muestra la serie de tiempo
fig_time_series.show()

##### Viajes en Marzo del 2022


###### Cantidad de viajes cumplidos por día en Marzo del 2022

In [ ]:
# Agrupa los viajes cumplidos por fecha y cuenta la frecuencia
frecuencia_por_fecha = marzo_2023_df_cumplido.groupBy("DATE").count().orderBy("DATE")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_dias_marzo = frecuencia_por_fecha.toPandas()

# Crea el histograma con Plotly y establece las etiquetas en el eje x
fig = px.bar(data_pandas_dias_marzo, x="DATE", y="count", title="Histograma de la cantidad de Viajes Cumplidos por día en marzo de 2023 (APP)")

# Muestra el histograma
fig.show()

In [ ]:
# Genera una secuencia de fechas para el eje x
fechas = pd.date_range(start="2023-3-01", end="2023-3-31")

# Crea la línea de tiempo con Plotly
fig = px.line(data_pandas_dias_marzo, x=fechas, y="count", title="Línea de Tiempo de la cantidad de Viajes Cumplidos en marzo de 2023 (APP)")

# Personaliza el formato de las etiquetas del eje x
fig.update_xaxes(
    tickvals=fechas,
    ticktext=[fecha.strftime('%Y-%m-%d') for fecha in fechas],
    tickangle=45,
)

# Muestra la línea de tiempo
fig.show()

###### Cantidad de viajes semanales en Marzo del 2022

In [ ]:
# Agrupa los viajes cumplidos por el día de la semana y cuenta la frecuencia
frecuencia_por_dia_semana = marzo_2023_df_cumplido.groupBy("DAYOFWEEK").count().orderBy("DAYOFWEEK")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_semanal_marzo = frecuencia_por_dia_semana.toPandas()

# Mapea los números de día de la semana a sus nombres correspondientes
dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
data_pandas_semanal_marzo['DAYOFWEEK'] = data_pandas_semanal_marzo['DAYOFWEEK'].map({i: dia for i, dia in enumerate(dias_semana, start=1)})

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig = px.bar(data_pandas_semanal_marzo, x="DAYOFWEEK", y="count", title="Histograma de la cantidad de viajes Cumplidos en marzo de 2023 (APP)")
fig.update_xaxes(title_text="Día de la Semana", categoryorder='array', categoryarray=dias_semana)

# Muestra el histograma
fig.show()

In [ ]:
# Crea una línea de tiempo con puntos en cada día de la semana
fig_timeline = px.line(data_pandas_semanal_marzo, x="DAYOFWEEK", y="count", title="Línea de Tiempo de la cantidad de viajes Cumplidos por Día de la Semana (APP)")

# Personaliza los ejes
fig_timeline.update_xaxes(title_text="Día de la Semana")
fig_timeline.update_yaxes(title_text="Viajes")

# Agrega puntos a la línea de tiempo
fig_timeline.update_traces(mode="markers+lines", marker=dict(size=8))

# Muestra la línea de tiempo
fig_timeline.show()

###### Cantidad de viajes cumplidos por hora en Marzo del 2023

In [ ]:
# Agrupa los viajes cumplidos por la hora completa del día y cuenta la frecuencia
frecuencia_por_hora_completa = marzo_2023_df_cumplido.groupBy("HOUR").count().orderBy("HOUR")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_horas_marzo = frecuencia_por_hora_completa.toPandas()

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig_hour_completa = px.bar(data_pandas_horas_marzo, x="HOUR", y="count", title="Histograma de cantidad de Viajes Cumplidos por en marzo de 2023 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_hour_completa.update_xaxes(title_text="Hora Completa del Día", tickmode='linear', tick0=0, dtick=1)

# Muestra el histograma
fig_hour_completa.show()

In [ ]:
# Crea la serie de tiempo con Plotly
fig_time_series = px.line(data_pandas_horas_marzo, x="HOUR", y="count", title="Serie de Tiempo de la cantidad de Viajes Cumplidos por hora en marzo de 2023 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_time_series.update_xaxes(title_text="Hora Completa del Día", tickmode='array', tickvals=data_pandas_horas_marzo['HOUR'], ticktext=data_pandas_horas_marzo['HOUR'])

# Agrega puntos para identificar mejor los datos
fig_time_series.update_traces(mode="markers+lines", marker=dict(size=4))

# Muestra la serie de tiempo
fig_time_series.show()

##### Viajes en Abril del 2022


###### Cantidad de viajes cumplidos por día en Abril del 2022

In [ ]:
# Agrupa los viajes cumplidos por fecha y cuenta la frecuencia
frecuencia_por_fecha = abril_2023_df_cumplido.groupBy("DATE").count().orderBy("DATE")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_dias_abril = frecuencia_por_fecha.toPandas()

# Crea el histograma con Plotly y establece las etiquetas en el eje x
fig = px.bar(data_pandas_dias_abril, x="DATE", y="count", title="Histograma de la cantidad de Viajes Cumplidos por día en abril de 2023 (APP)")

# Muestra el histograma
fig.show()

In [ ]:
# Genera una secuencia de fechas para el eje x
fechas = pd.date_range(start="2023-4-01", end="2023-4-30")

# Crea la línea de tiempo con Plotly
fig = px.line(data_pandas_dias_abril, x="DATE", y="count", title="Línea de Tiempo de la cantidad de Viajes Cumplidos en abril de 2023 (APP)")

# Personaliza el formato de las etiquetas del eje x
fig.update_xaxes(
    tickvals=fechas,
    ticktext=[fecha.strftime('%Y-%m-%d') for fecha in fechas],
    tickangle=45,
)

# Muestra la línea de tiempo
fig.show()

###### Cantidad de viajes semanales en Abril del 2022

In [ ]:
# Agrupa los viajes cumplidos por el día de la semana y cuenta la frecuencia
frecuencia_por_dia_semana = abril_2023_df_cumplido.groupBy("DAYOFWEEK").count().orderBy("DAYOFWEEK")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_semanal_abril = frecuencia_por_dia_semana.toPandas()

# Mapea los números de día de la semana a sus nombres correspondientes
dias_semana = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
data_pandas_semanal_abril['DAYOFWEEK'] = data_pandas_semanal_abril['DAYOFWEEK'].map({i: dia for i, dia in enumerate(dias_semana, start=1)})

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig = px.bar(data_pandas_semanal_abril, x="DAYOFWEEK", y="count", title="Histograma de la cantidad de viajes Cumplidos en abril de 2023 (APP)")
fig.update_xaxes(title_text="Día de la Semana", categoryorder='array', categoryarray=dias_semana)

# Muestra el histograma
fig.show()

In [ ]:
# Crea una línea de tiempo con puntos en cada día de la semana
fig_timeline = px.line(data_pandas_semanal_abril, x="DAYOFWEEK", y="count", title="Línea de Tiempo de la cantidad de viajes Cumplidos por Día de la Semana (APP)")

# Personaliza los ejes
fig_timeline.update_xaxes(title_text="Día de la Semana")
fig_timeline.update_yaxes(title_text="Viajes")

# Agrega puntos a la línea de tiempo
fig_timeline.update_traces(mode="markers+lines", marker=dict(size=8))

# Muestra la línea de tiempo
fig_timeline.show()

###### Cantidad de viajes cumplidos por hora en Abril del 2022

In [ ]:
# Agrupa los viajes cumplidos por la hora completa del día y cuenta la frecuencia
frecuencia_por_hora_completa = abril_2023_df_cumplido.groupBy("HOUR").count().orderBy("HOUR")

# Convierte el DataFrame de Spark en un DataFrame de pandas para Plotly
data_pandas_horas_abril = frecuencia_por_hora_completa.toPandas()

# Crea el histograma con Plotly y personaliza las etiquetas del eje X
fig_hour_completa = px.bar(data_pandas_horas_abril, x="HOUR", y="count", title="Histograma de cantidad de Viajes Cumplidos por en abril de 2023 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_hour_completa.update_xaxes(title_text="Hora Completa del Día", tickmode='linear', tick0=0, dtick=1)

# Muestra el histograma
fig_hour_completa.show()

In [ ]:
# Crea la serie de tiempo con Plotly
fig_time_series = px.line(data_pandas_horas_abril, x="HOUR", y="count", title="Serie de Tiempo de la cantidad de Viajes Cumplidos por hora en abril de 2023 (APP)")

# Personaliza las etiquetas del eje X para mostrar todas las horas
fig_time_series.update_xaxes(title_text="Hora Completa del Día", tickmode='array', tickvals=data_pandas_horas_abril['HOUR'], ticktext=data_pandas_horas_abril['HOUR'])

# Agrega puntos para identificar mejor los datos
fig_time_series.update_traces(mode="markers+lines", marker=dict(size=4))

# Muestra la serie de tiempo
fig_time_series.show()

##### Comparativa lineas de tiempo por dias del mes


In [ ]:
import plotly.graph_objects as go

# Genera una secuencia de fechas para el eje x que cubra todo el rango
fechas_completas = pd.date_range(start="2022-10-01", end="2023-04-30")

# Crear un diccionario con los datos de frecuencia por mes
datos_meses = {
    "Octubre 2022": data_pandas_dias_octubre,
    "Noviembre 2022": data_pandas_dias_noviembre,
    "Diciembre 2022": data_pandas_dias_diciembre,
    "Enero 2023": data_pandas_dias_enero,
    "Febrero 2023": data_pandas_dias_febrero,
    "Marzo 2023": data_pandas_dias_marzo,
    "Abril 2023": data_pandas_dias_abril
}

# Crear una figura Plotly para la gráfica
fig = go.Figure(layout_title="Comparativa de las lineas de Tiempo de la Cantidad de Viajes Cumplidos por (APP)")

# Personalizar el formato de las etiquetas del eje x
fig.update_xaxes(tickvals=fechas_completas[::10], ticktext=fechas_completas[::10].strftime('%Y-%m-%d'), tickangle=45)

# Agrega las líneas de tiempo con etiquetas diferentes
for mes, datos in datos_meses.items():
    fig.add_trace(go.Scatter(x=datos["DATE"], y=datos["count"], mode="lines", name=mes))

# Muestra la gráfica
fig.show()

El gráfico muestra cómo la cantidad de viajes cumplidos en una aplicación (APP) varía a lo largo de varios meses, desde octubre de 2022 hasta abril de 2023. Cada línea representa un mes distinto y se etiqueta con el nombre del mes.

Se puede observar que la cantidad de viajes varía significativamente de un mes a otro.
Algunos meses tienen un mayor número de viajes, mientras que otros meses muestran menos actividad.
La visualización facilita la comparación entre los meses.

##### Comparativa lineas de tiempo por viajes semanales


In [ ]:
# Crear un diccionario con los datos de frecuencia semanal por mes
datos_semanales_meses = {
    "Octubre 2022": data_pandas_semanal_octubre,
    "Noviembre 2022": data_pandas_semanal_noviembre,
    "Diciembre 2022": data_pandas_semanal_diciembre,
    "Enero 2023": data_pandas_semanal_enero,
    "Febrero 2023": data_pandas_semanal_febrero,
    "Marzo 2023": data_pandas_semanal_marzo,
    "Abril 2023": data_pandas_semanal_abril
}

# Crear una figura Plotly para la gráfica
fig = go.Figure(layout_title="Comparativa de las Líneas de Tiempo de la Cantidad de Viajes Cumplidos por (APP) (Formato Semanal)")

# Personalizar el formato de las etiquetas del eje x
dias_semana = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]
fig.update_xaxes(tickvals=list(range(7)), ticktext=dias_semana, tickangle=0)

# Agregar las líneas de tiempo semanales con etiquetas diferentes para cada mes
for mes, datos_semanales in datos_semanales_meses.items():
    # Agrupar los datos por día de la semana y obtener el total
    total_semanal = datos_semanales.groupby(datos_semanales.DAYOFWEEK)["count"].sum()
    fig.add_trace(go.Scatter(x=list(range(7)), y=total_semanal, mode="lines", name=mes))

# Muestra la gráfica
fig.show()

Esta gráfica compara la cantidad de viajes cumplidos por día de la semana para diferentes meses, desde octubre de 2022 hasta abril de 2023. Esto es útil en la identificación de patrones estacionales y tendencias en la demanda de viajes.

* Lunes:

  * El mes con el mayor número de viajes los lunes fue octubre de 2022, con un pico destacado.
  * Abril de 2023 muestra el menor número de viajes los lunes.

* Martes:

  * El mes con la mayor cantidad de viajes los martes fue marzo de 2023.
  * Abril de 2023 registra la menor cantidad de viajes los martes.

* Miércoles:

  * Octubre de 2022 lidera en la cantidad de viajes los miércoles.
  * Abril de 2023 exhibe la menor cantidad de viajes los miércoles.

* Jueves:

  * Noviembre de 2022 se destaca con la mayor cantidad de viajes los jueves.
  * Abril de 2023 presenta la menor cantidad de viajes los jueves.

* Viernes:

  * Noviembre de 2022 lidera en la cantidad de viajes los viernes.
  * Abril de 2023 muestra la menor cantidad de viajes los viernes.

* Sábado:

  * El mes con la mayor cantidad de viajes los sábados fue octubre de 2022.
  * Abril de 2023 registra la menor cantidad de viajes los sábados.

* Domingo:

  * Marzo de 2023 sobresale con la mayor cantidad de viajes los domingos.
  * Abril de 2023 tiene la menor cantidad de viajes los domingos.

##### Comparativa lineas de tiempo por hora


In [ ]:
# Crear un diccionario con los datos de frecuencia horaria desde octubre de 2022 hasta abril de 2023
datos_horarios_meses = {
    "Octubre 2022": data_pandas_horas_octubre,
    "Noviembre 2022": data_pandas_horas_noviembre,
    "Diciembre 2022": data_pandas_horas_diciembre,
    "Enero 2023": data_pandas_horas_enero,
    "Febrero 2023": data_pandas_horas_febrero,
    "Marzo 2023": data_pandas_horas_marzo,
    "Abril 2023": data_pandas_horas_abril
}

# Crear una figura Plotly para la gráfica
fig = go.Figure(layout_title="Comparativa de las Líneas de Tiempo de la Cantidad de Viajes Cumplidos por (APP) (Formato Horario)")

# Personalizar el formato de las etiquetas del eje x
horas_del_dia = list(range(24))
fig.update_xaxes(tickvals=horas_del_dia, ticktext=[str(hora) + ":00" for hora in horas_del_dia], tickangle=0)

# Agregar las líneas de tiempo horarias con etiquetas diferentes para cada mes
for mes, datos_horarios in datos_horarios_meses.items():
    # Agrupar los datos por hora del día y obtener el total
    total_horario = datos_horarios.groupby(datos_horarios["HOUR"])["count"].sum()
    fig.add_trace(go.Scatter(x=horas_del_dia, y=total_horario, mode="lines", name=mes))

# Muestra la gráfica
fig.show()


La gráfica compara la cantidad de viajes cumplidos por hora en diferentes meses, desde octubre de 2022 hasta abril de 2023. Aquí hay un análisis conciso de las tendencias de la demanda de viajes:

En general, se observa un patrón común en la mayoría de los meses:

  * Durante las primeras horas del día, entre la medianoche y las 7:00 AM, hay una disminución en la cantidad de viajes.
  * La demanda de viajes comienza a aumentar gradualmente a medida que avanza la mañana desde las 7:00 AM hasta las 11:00 AM, alcanzando su pico.
  * Luego la cantidad de viajes comienza a disminuir hasta alrededor de las 4:00 PM para nuevamente subir gradualmente hasta las 11:00 PM

Este análisis revela variaciones en la demanda de viajes a lo largo del día y entre los meses. Comprender estas tendencias es esencial para la optimización de servicios de transporte y la asignación de recursos en función de las necesidades cambiantes de los usuarios a lo largo del tiempo.